In [26]:
import cudf as pd
import cupy as np
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import Sequential, models
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D

import matplotlib.pyplot as plt

In [27]:
path = "../input/face-mask-12k-images-dataset/Face Mask Dataset/"

In [28]:
dataset = {
    "image_path": [],
    "mask_status": [],
    "where": []
}

for where in os.listdir(path):
    for status in os.listdir(path+"/"+where):
        for image in glob.glob(path+where+"/"+status+"/"+"*.png"):
            dataset["image_path"].append(image)
            dataset["mask_status"].append(status)
            dataset["where"].append(where)
            
df = pd.DataFrame(dataset)
df.head()

In [29]:
# Data visualisation
print(df.groupby("mask_status").count())

In [30]:
# Splitting train, test and Validation Dataset
train_df = df[df["where"] == "Train"]
test_df = df[df["where"] == "Test"]
valid_df = df[df["where"] == "Validation"]
print("\n Before Shuffling \n")
print(train_df.head(10))

# Shuffling the dataset(Shuffling data reduces variance and leads to less overfitting)
train_df = train_df.sample(frac = 1)
test_df = test_df.sample(frac = 1)
valid_df = valid_df.sample(frac = 1)

print("\n After Shuffling \n")
print(train_df.head(10))

In [31]:
train_df = train_df.reset_index().drop("index", axis = 1)
train_df.head()

In [32]:
# Resizing images
data = []
image_size = 150

for i in range(len(train_df)):
    img_array = cv2.imread(train_df["image_path"][i], cv2.IMREAD_GRAYSCALE)
    # Resizing the array
    new_image_array = cv2.resize(img_array, (image_size, image_size))

    # Label Encoding
    if train_df["mask_status"][i] == "WithMask":
        data.append([new_image_array, 1])
    else:
        data.append([new_image_array, 0])

In [33]:
X = []
y = []

# Seperating X and y
for image in data:
    X.append(image[0])
    y.append(image[1])

X = np.array(X)
y = np.array(y)
X = np.asnumpy(X)
y = np.asnumpy(y)

In [34]:
X = X / 255 # Data Normalization
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [35]:
# Build the model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPool2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [36]:
#Compiling the model
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [37]:
X_train = X_train.reshape(len(X_train), X_train.shape[1], X_train.shape[2], 1)
X_val = X_val.reshape(len(X_val), X_val.shape[1], X_val.shape[2], 1)
history = model.fit(X_train, y_train, epochs=5, batch_size = 32)

In [38]:
model.summary()

In [39]:
test_loss,test_acc = model.evaluate(X_val, y_val)
print("Test loss: ",test_loss)
print("Test accuracy: ",test_acc*100)